<a href="https://colab.research.google.com/github/Mshafei2010/Twitter-Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

## downlaods

In [ ]:
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
!pip install emot
!pip install contractions

In [74]:
import nltk
from nltk.corpus import words
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
import emot
import contractions
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import random_split, TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.feature_extraction.text import TfidfVectorizer

# Fetching Data

In [ ]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

## Reading Data

In [ ]:
positive_tweets=twitter_samples.strings('positive_tweets.json')
negative_tweets=twitter_samples.strings('negative_tweets.json')

## Explore Data

In [ ]:
print(positive_tweets[0])
print(negative_tweets[0])

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
hopeless for tmr :(


# Preprocess Data

In [209]:
stop_words = set(stopwords.words('english'))
stop_words.add("us")
english_wordss = set(words.words())

In [210]:
def delete_and_append(string, beg_index, end_index, char):
  new_string = string[:beg_index] + char + string[end_index + 1:]
  return new_string

In [215]:
emot_obj = emot.core.emot()
def preprocess_tweet(tweet):
  #I'm -> I am
  tweet = contractions.fix(tweet)
  #emotions
  if emot_obj.emoticons(tweet)['location']!=[] or emot_obj.emoticons(tweet)['mean']!=[]:
    tweet_with_emo = delete_and_append(tweet,emot_obj.emoticons(tweet)['location'][0][0],emot_obj.emoticons(tweet)['location'][0][0],emot_obj.emoticons(tweet)['mean'][0].split()[0])
  else:
    tweet_with_emo=tweet
  #lowercase
  tweet_with_emo=tweet_with_emo.lower()
  #hashtag
  tweet_hastag_free= re.sub(r'#\w+', '', tweet_with_emo)
  #mentions
  tweet_mentions_free= re.sub(r'@\w+', '', tweet_hastag_free)
  #spaces
  hastag_mentions_free=tweet_mentions_free.lstrip(' ')
  #get each word
  words = word_tokenize(hastag_mentions_free)
  #filter from stopwords
  filtered_words = [word for word in words if word not in stop_words]
  #make limitize for for words
  lemmatizer=WordNetLemmatizer()
  #verb
  lemmatized_words=[lemmatizer.lemmatize(word,pos='v') for word in filtered_words]
  #adjective "better" → "good"
  lemmatized_words=[lemmatizer.lemmatize(word,pos='a') for word in lemmatized_words]
  #adverbs "best" → "well"
  lemmatized_words=[lemmatizer.lemmatize(word,pos='r') for word in lemmatized_words]
  #remove plural
  lemmatized_words=[lemmatizer.lemmatize(word,pos='s') for word in lemmatized_words]
  #noun
  #ERRRRRORRRRR us -> u ------> fixed by adding us
  lemmatized_words=[lemmatizer.lemmatize(word,pos='n') for word in lemmatized_words]
  #remove special charcters and spaces
  filtered_word = [re.sub(r'[^\w\s]', '', word).strip() for word in lemmatized_words if re.sub(r'[^\w\s]', '', word).strip() != '']
  #filter digits
  filtered_word = [word for word in filtered_word if not word.isdigit()]
  #remove words with numbers
  filtered_word = [word for word in filtered_word if not any(char.isdigit() for char in word)]
  #remove non english
  filtered_word = [word for word in filtered_word if word.isalpha()]
  filtered_word = [word for word in filtered_word if word in english_wordss]
  return filtered_word

In [216]:
result = preprocess_tweet(negative_tweets[0])
print(result)

['hopeless', 'tomorrow', 'frown']


In [217]:
df = pd.DataFrame(columns=['tweet words', 'label'])
for tweet in positive_tweets:
  result = preprocess_tweet(tweet)
  if result!=[]:
    df.loc[len(df.index)] = [result, 1]
for tweet in negative_tweets:
  result = preprocess_tweet(tweet)
  if result!=[]:
    df.loc[len(df.index)] = [result, 0]
df.head()

,tweet words,label
0,"[top, engage, member, community, week, happy]",1
1,"[hey, jam, odd, skeptical, please, call, conta...",1
2,"[listen, last, night, happy, bleed, amaze, track]",1
3,[happy],1
4,"[verify, succeed, get, blue, tick, mark, profi...",1


In [218]:
print(df.shape)

(9971, 2)


# Machine Learning

1.   Text Embedding
2.   Machine Learning Model



## Text Embedding

### Doc2Vec

In [ ]:
Doc2vec_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(df['tweet words'])]
print(Doc2vec_data)

In [50]:
doc2vec_model = Doc2Vec(vector_size=100, min_count=1, window = 5 , epochs=100)
doc2vec_model.build_vocab(Doc2vec_data)
doc2vec_model.train(Doc2vec_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

print(doc2vec_model.dv[0])

[-0.0259928  -0.1414958  -0.06825285 -0.0238857  -0.01215556 -0.3022812
 -0.03255841  0.18243347 -0.12728953  0.01848797 -0.09998083 -0.01098354
 -0.04868787 -0.04161329  0.07414528 -0.09222618  0.26297265  0.01219682
  0.09501714 -0.19942158 -0.00532975 -0.05247219 -0.00977621  0.04027597
  0.12564792  0.11593635 -0.1030084  -0.05527261  0.11096427  0.00213732
  0.3282094   0.05465385 -0.05678825 -0.05746384  0.10632078  0.12865451
  0.0915288  -0.20398347  0.18443775 -0.14767827 -0.07917891 -0.27900735
 -0.05137599  0.09323786  0.05290314  0.11004949 -0.18363628  0.08053365
 -0.11762113  0.07007325  0.14783034  0.00467675  0.12157117  0.1800047
 -0.20201768  0.08890102  0.11784799  0.03513597 -0.14623584  0.09913978
 -0.00728375 -0.0921988  -0.01669612 -0.02353666 -0.1383095   0.22656779
  0.01364232  0.05121407 -0.28077158  0.22292581 -0.28289405  0.10056175
  0.1248244   0.17152247  0.03314182  0.03903512 -0.24483943  0.1615229
 -0.01259523  0.04587504 -0.18234825  0.08591085 -0.15

## Machine Learning Model

In [55]:
print(len(df['tweet words']))
doc_vectors = [doc2vec_model.dv[i] for i in range(len(df['tweet words']))]
# print(doc_vectors)


X_train, X_test, y_train, y_test = train_test_split(doc_vectors, df['label'], test_size=0.2, random_state=42)

9971


In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

classifier = LogisticRegression()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8546365914786967


# Deep Learning



## Word Embedding

### TF-IDF

In [331]:
def TF_id(all_tweets_lists):
  TFvectorizer = TfidfVectorizer()
  tweets_matrix = TFvectorizer.fit_transform(all_tweets_lists)
  TFdf = pd.DataFrame(tweets_matrix.toarray(), columns=TFvectorizer.get_feature_names_out())
  TFdf.set_index([pd.Index(all_tweets_lists)],inplace=True)
  return TFdf

In [332]:
TFtweets = [' '.join(words) for words in df['tweet words']]
TFdf = TF_id(TFtweets)
TFdf.head()

,aa,abandon,able,aboard,abroad,abrupt,abscess,absent,absolute,absolutely,...,yuck,yummy,zac,zante,zero,zipper,zodiac,zone,zoo,zoom
top engage member community week happy,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hey jam odd skeptical please call contact able assist many thank,0.0,0.0,0.292945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
listen last night happy bleed amaze track,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
happy,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
verify succeed get blue tick mark profile happy day,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## RNN

In [426]:
class SimpleRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(SimpleRNN, self).__init__()
    self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
    self.rnn1 = nn.RNN(hidden_size, int(hidden_size/2), batch_first=True)
    self.fc = nn.Linear(int(hidden_size/2), output_size)

  def forward(self, x):
    h0 = torch.zeros(1, x.size(0), self.rnn.hidden_size).to(x.device)
    out, _ = self.rnn(x,h0)
    h1 = torch.zeros(1, x.size(0), self.rnn1.hidden_size).to(x.device)
    out, _ = self.rnn1(out,h1)
    out = self.fc(out[:, -1, :])
    out = torch.sigmoid(out)
    return out

In [427]:
print(TFdf.shape[1])
input_size = TFdf.shape[1]
hidden_size = 32
output_size = 1
model = SimpleRNN(input_size, hidden_size, output_size)

4621


In [428]:
feature_tensor = torch.tensor(TFdf.values, dtype=torch.float32)
label_tensor = torch.tensor(df['label'], dtype=torch.float32)

dataset = TensorDataset(feature_tensor, label_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [429]:
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.001)

num_epochs = 15
model.train()

for epoch in range(num_epochs):
  for batch_features, batch_labels in train_loader:
    batch_features = batch_features.view(batch_features.size(0), 1, -1)
    outputs = model(batch_features)
    loss = criterion(outputs.squeeze(), batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/15], Loss: 0.0613
Epoch [2/15], Loss: 0.0145
Epoch [3/15], Loss: 0.0334
Epoch [4/15], Loss: 0.0011
Epoch [5/15], Loss: 0.0008
Epoch [6/15], Loss: 0.0007
Epoch [7/15], Loss: 0.0007
Epoch [8/15], Loss: 0.0007
Epoch [9/15], Loss: 0.0001
Epoch [10/15], Loss: 0.0002
Epoch [11/15], Loss: 0.0001
Epoch [12/15], Loss: 0.0001
Epoch [13/15], Loss: 0.0001
Epoch [14/15], Loss: 0.0003
Epoch [15/15], Loss: 0.3606


In [433]:
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for batch_features, batch_labels in test_loader:
    batch_features = batch_features.view(batch_features.size(0), 1, -1)
    outputs = model(batch_features)
    predicted = (outputs > 0.5).int()
    total += batch_labels.size(0)
    correct += (predicted.squeeze() == batch_labels).sum().item()
  accuracy = 100 * correct / total
  print(f'Accuracy on test set: {accuracy:.2f}%')

Accuracy on test set: 97.39%
